In [4]:
#necessary imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, balanced_accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import InputLayer, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.math import confusion_matrix as tf_confusion_matrix

In [5]:
dummy_plays = pd.read_csv('/dummy_plays.csv')
dummy_plays.head(3)

,Unnamed: 0,OffenseTeam,DefenseTeam,ToGo,PlayType,DefensePassRank,DefenseRunRank,Down_1,Down_2,Down_3,Down_4,Formation_NO HUDDLE,Formation_NO HUDDLE SHOTGUN,Formation_SHOTGUN,Formation_UNDER CENTER
0,0,LV,MIA,19,0,25,16,0,0,1,0,0,0,1,0
1,1,MIN,NO,1,0,5,5,1,0,0,0,0,0,0,1
2,2,TB,DET,6,1,32,14,0,1,0,0,0,1,0,0


In [6]:
dummy_plays.drop(columns='Unnamed: 0',inplace=True)

In [7]:
run_rates = []
pass_rates = []
tf_accuracy_scores = []

for name in set(dummy_plays['OffenseTeam']):
    team_data = dummy_plays[dummy_plays['OffenseTeam'] == name]
    X = team_data.drop(columns=['OffenseTeam','DefenseTeam','PlayType'])
    y = team_data['PlayType']
    run_rate = team_data['PlayType'].value_counts(normalize=True).sort_values(ascending=True)[0]
    run_rates.append(run_rate)
    pass_rate = team_data['PlayType'].value_counts(normalize=True).sort_values(ascending=True)[1]
    pass_rates.append(pass_rate)
    X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=42)
    ss = StandardScaler()
    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)
    model = Sequential()
    model.add(InputLayer(input_shape=(11,)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(32,activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(.05))
    model.add(Dense(16,activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(.20))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='adam',loss='bce', metrics=['accuracy'])
    
    callback = EarlyStopping(monitor='val_loss', patience=3)

    results = model.fit(X_train_sc,y_train,epochs=50,validation_data=(X_test_sc,y_test), callbacks=[callback])
    
    tf_accuracy_scores.append(model.evaluate(X_test_sc,y_test))

acc_score_df = pd.DataFrame(set(dummy_plays['OffenseTeam']))
acc_score_df['tf_accuracy_scores'] = tf_accuracy_scores
acc_score_df.columns = ['Team','tf_accuracy_scores']
acc_score_df['Run_Rate'] = run_rates
acc_score_df['Pass_Rate'] = pass_rates
team_scores = acc_score_df.sort_values(by='tf_accuracy_scores')

Epoch 1/50
23/23 [==============================] - 1s 10ms/step - loss: 1.0964 - accuracy: 0.5347 - val_loss: 1.0612 - val_accuracy: 0.6292
Epoch 2/50
23/23 [==============================] - 0s 3ms/step - loss: 1.0295 - accuracy: 0.5931 - val_loss: 1.0021 - val_accuracy: 0.6417
Epoch 3/50
23/23 [==============================] - 0s 2ms/step - loss: 0.9714 - accuracy: 0.6375 - val_loss: 0.9547 - val_accuracy: 0.6375
Epoch 4/50
23/23 [==============================] - 0s 3ms/step - loss: 0.9280 - accuracy: 0.6361 - val_loss: 0.9161 - val_accuracy: 0.6458
Epoch 5/50
23/23 [==============================] - 0s 3ms/step - loss: 0.8886 - accuracy: 0.6389 - val_loss: 0.8814 - val_accuracy: 0.6375
Epoch 6/50
23/23 [==============================] - 0s 3ms/step - loss: 0.8582 - accuracy: 0.6528 - val_loss: 0.8512 - val_accuracy: 0.6375
Epoch 7/50
23/23 [==============================] - 0s 4ms/step - loss: 0.8236 - accuracy: 0.6639 - val_loss: 0.8239 - val_accuracy: 0.6292
Epoch 8/50
23/23 [=

In [8]:
team_scores['tf_accuracy_scores'] = [i[1] for i in team_scores['tf_accuracy_scores']]

In [9]:
team_scores['tf_accuracy_scores'].sum()

23.16356670856476

In [10]:
team_scores.sort_values('tf_accuracy_scores',ascending=False)

,Team,tf_accuracy_scores,Run_Rate,Pass_Rate
14,LA,0.811245,0.424547,0.575453
18,NYJ,0.787736,0.431953,0.568047
2,DET,0.782805,0.375991,0.624009
31,CAR,0.780822,0.399543,0.600457
24,PIT,0.767347,0.345957,0.654043
9,JAX,0.765766,0.324294,0.675706
4,TB,0.757576,0.360478,0.639522
7,ATL,0.754098,0.382564,0.617436
13,MIA,0.747788,0.404867,0.595133
27,TEN,0.739316,0.498396,0.501604
